<a href="https://colab.research.google.com/github/sizhky/naive-bayes-demo/blob/main/spam-ham/step_1_first_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install torch_snippets

In [ ]:
## Setup and imports
import os
from pathlib import Path
import urllib.request
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_snippets import unzip_file

2021-07-29 05:57:58.436 | WARNING  | torch_snippets.torch_loader:<module>:233 - Not importing Lightning Report
2021-07-29 05:57:58.444 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'


In [ ]:
if not os.path.exists('SMSSpamCollection'):
    urllib.request.urlretrieve ("https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip", "smsspamcollection.zip")
    # Extracting the dataset
    unzip_file('smsspamcollection.zip', './')

In [10]:
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None)
df.columns = ['class','content']
trn_df, val_df = train_test_split(df, random_state=10)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

vectorizer = CountVectorizer()
vectorizer.fit(trn_df['content'])
trn = vectorizer.transform(trn_df['content']).todense()
val = vectorizer.transform(val_df['content']).todense()
words = vectorizer.get_feature_names()

nb = MultinomialNB()
nb.fit(trn, trn_df['class'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report
classes = ['ham','spam']

_y = nb.predict(val)
y = val_df['class']

print(confusion_matrix(y, _y))
print(classification_report(y, _y))

feature_probs = np.array(nb.feature_log_prob_)

k = 15
for i in range(len(feature_probs)):
    print(f'Top {k} features for {classes[i]}')
    _f = np.argsort(feature_probs[i])[-k:]
    print([words[j] for j in _f])

[[1209    6]
 [  19  159]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1215
        spam       0.96      0.89      0.93       178

    accuracy                           0.98      1393
   macro avg       0.97      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393

Top 15 features for ham
['can', 'not', 'have', 'of', 'for', 'that', 'it', 'is', 'my', 'me', 'in', 'and', 'the', 'to', 'you']
Top 15 features for spam
['from', 'ur', 'have', 'on', 'is', 'txt', 'or', 'now', 'for', 'the', 'free', 'your', 'you', 'call', 'to']
